**By Peter A. Stokes, École Pratique des Hautes Études – Université PSL**

_These are brief notes and exercises on working with TEI XML files using Python. They are intended as teaching aids for the course on 'Image Processing with Python' which is part of the Atelier de formation annuel du Consortium Cahier on the topic of 'Exploiter les corpus d'auteurs' in Poitiers, 18--20 June 2019. For more details see https://cahier.hypotheses.org/4662_

These notes assume a good knowledge of TEI XML but assume **no experience or knowledge at all** in programming. This notebook also assumes that the [Python Imaging Library (PIL)](https://pythonware.com/products/pil/) has already been installed in your Python system, and that an adapted version of the file [here](http://xtf.bvh.univ-tours.fr/xtf/data/tei/B330636101_S1238/B330636101_S1238_tei.xml) has already been downloaded and saved in the same directory as this notebook.

_If you are viewing this in Jupyter then you can edit the code simply by typing in the boxes. You can also execute the code in any box by clicking on the box and typing SHIFT + ENTER or using the 'Run' button in the menubar above._

# **\[Ad intro/context\]**

1. IIIF: NB relationship between TEI XML @facs and the IIIF, e.g. https://gallica.bnf.fr/iiif/ark:/12148/bpt6k11718168/f17/full/pct:25/0/native.jpg
   * So could easily load images with IIIF
   * Could also use TEI documentary view to load sections of images with Python & IIIF
   
**Remember: already have slides on IIIF!**

# Getting the Image Addresses

In order to manipulate the image, we first need to get all the addresses. This is the procedure we want to follow:

1. `import` the Beautiful Soup library
1. `open()` the _Essais_ file and store the result in a variable
1. `read()` the file into another variable
1. Process the results of the previous step using `BeautifulSoup` with `"xml"` format and save the results in a new variable
1. `close()` the file
1. `select()` all the `pb` elements and store the resulting list in a variable
1. Loop `for` each pb element `in` the list variable:
   1. `print()` the value of the `"facs"` attribute
   
Try to write the code to do this yourself. You will want to look back at material from the previous sessions, particularly [1. Getting Started with XML in Python](1.%20Getting%20Started%20with%20XML%20in%20Python.ipynb) 

In [ ]:
import urllib, json, io, requests
from PIL import Image

manifest_url = "https://gallica.bnf.fr/iiif/ark:/12148/bpt6k11718168/manifest.json"

conn = urllib.request.urlopen(manifest_url)

manifest = json.loads(conn.read())

Now we read the manifest file to find the URLs of the images. The IIIF is a relatively complex file, so it's not very obvious how to extract the images from it. In principle, though, it's structured as follows:

1. Each manuscript can have one or more sequences of pages. This allows different page orders to be stored, for instance the current and original page ordering. In practice the vast majority of manifests have only one sequence, so we just take the first.
1. Each sequence then contains a number of 'canvases'. In practice, for us, each canvas corresponds to a page, so this is effectively the list of pages.
1. We then go through and loop `for` each `canvas` in our list of canvases:
   1. We get the list of images for each canvas. In principle we can have more than one image but again in practice this is very rare, so we just take the first image record in the list.
   1. The `image` record contains a `resource` record, and the `resource` record contains the `@id` field. The `@id` is in fact is the full URL to our image, so we save it.
   1. We now print out the address for the URL
   
**You must be very careful when looping like this**. It's easy to make a mistake and suddenly send hundreds or thousands of download requests to the BnF website. This is likely to get you in trouble, as it could be interpreted as a common hacking activity (a ['denial of service'](https://en.wikipedia.org/wiki/Denial-of-service_attack) or DOS attack). This is likely to mean you will be blocked from the BnF website, and could theoretically even result in a crimina investigation! For this reason, we test our results first by printing out to the screen and making sure that our code is working. Only then do we really connect to the actual website.

In [ ]:
# Get the sequence of pages from the manifest
sequence = manifest["sequences"][0]

# Get the list of pages ('canvases') from the sequence
canvases = sequence["canvases"]

# Now go through the list of canvases
for canvas in canvases[20:25]:
    im_addr = canvas["images"][0]["resource"]["@id"]
    print(im_addr)

You should get five URLs, and you can click on them to make sure that the addresses are correct.

You will notice that the images are very large and very high quality. This is nice but we really don't need such high quality for our work. We want to be good citizens and not load the BnF site any more than we need to, so we can change the URL to tell the IIIF server that we only want an image at 25% of full size. To do this, we need to change the URLs of each address, specifically, the part that says `/full/full/0` we want to change to `/full/pct:25/0` (for '25 percent'). Fortunately this is easy to do in Python using the `replace()` method. Here is how it works:

In [ ]:
for canvas in canvases[20:25]:
    # Get the full-resolution address of the image from the canvas record
    im_addr = canvas["images"][0]["resource"]["@id"]
    
    # Change the resolution from 'full' to 25%
    im_addr = im_addr.replace("/full/full/0", "/full/pct:25/0")
    
    # Download the image and store it in a PIL Image variable
    res = requests.get(im_addr)
    image = Image.open(io.BytesIO(res.content))
    
    # Show the image to be sure it worked
    image.show()

This is good, but it's very inefficient to download the image every time we want to use it. Instead, we can save a copy in memory, by storing it in a variable. The problem here is that we have more than one image. We could store them in many different variables, such as `image1`, `image2` etc. but this is very inefficient (what happens if we change the number of images? what happens if we want to download hundreds of images? etc.). Instead, we can create a list of images, and simply add each new image to the list using the `append()` method. Here's how it works:

In [ ]:
# To begin, we have to create an empty list
images = []

# Now we go through our loop again
for canvas in canvases[20:25]:
    im_addr = canvas["images"][0]["resource"]["@id"]
    im_addr = im_addr.replace("/full/full/0", "/full/pct:25/0")
    res = requests.get(im_addr)
    image = Image.open(io.BytesIO(res.content))
    
    # Here, instead of showing the image, we add it to our list
    images.append(image)
    
# Now we should have a list of images.
# Let's test by looping through and showing each one
for image in images:
    print(image.size)
    image.show()

Now that we have our list, we can use it again and again to do different things with it. For instance, we can loop `for` each image `in` our list `images` and print the `size`. Try writing the code to do this into the box below:

Another interesting possibility of IIIF is that it allows us to access specific regions of the page (see the [IIIF specifications](https://iiif.io/api/image/2.1/#region) for more details). In this case, we have to change the first occurrence of 'full' in the URL, replacing it with the coordinates of the region we want. For instance, let's compare the following two URLs, one with the full image and one with a section. You should see two images, one of the full page and one of some text in line 7.

In [ ]:
url_full = "https://gallica.bnf.fr/iiif/ark:/12148/bpt6k11718168/f23/full/pct:25/0/native.jpg"

res = requests.get(url_full)
image = Image.open(io.BytesIO(res.content))
image.show()

url_region = "https://gallica.bnf.fr/iiif/ark:/12148/bpt6k11718168/f23/810,1250,2925,185/pct:25/0/native.jpg"

res = requests.get(url_region)
image = Image.open(io.BytesIO(res.content))
image.show()

For the exercise, let's now apply this same region to images of several different pages. To do this, we use the following algorithm:

1. Create a new empty list to hold our images
1. `for` each canvas `in` the list of canvases:
  1. Get the URL for the canvas and save it in a local variable
  1. Replace the full URL with the one for the region (and the reduced image size)
  1. Open the connection to the URL
  1. Read in the image data and create a new Image
  1. Add this new image to our list of images

Here is the code to do this:

In [ ]:
images2 = []

for canvas in canvases[20:25]:
    im_addr = canvas["images"][0]["resource"]["@id"]
    im_addr = im_addr.replace("/full/full/0", "/810,1250,2925,185/pct:25/0")
    res = requests.get(im_addr)
    image = Image.open(io.BytesIO(res.content))
    images2.append(image)

Now let's have a look at the images to make sure this worked:

In [ ]:
for image in images2:
    print(image.size)
    image.show()

What we've just done is nice, but frankly it's not very interesting. What would be much better is to have an image of each line of text on a given page. We can do this, but first we need to think a little bit about some basic arithmetic and geometry.

In order to get the image of a line of text, we need to calculate the coordinates of each line. This is relatively easy for a printed book, since the lines are very regular (compared to a notebook, for instance). Ideally we would use Python to automatically find the lines for us, but this is much more advanced (though we will see a simple version of this in the last 'Going Further' section of these notes). Instead of doing it automatically, we can do the following:

Using any imaging software, we need to measure the following:

1. The height of each line of text, in pixels
1. The width of each column, again in pixels
1. The x and y coordinates of the start of the first line of text

**Attention**: the Python Imaging Library (and indeed most computer imaging systems) measure the origin (coordinates 0,0) in the top left corner, and the y axis counts down. This is different from the 'standard' cartesian coordinate system that you may have learned in schoole, where the origin is in the middle and the y axis counts up.

Open the image yourself, measure the details and insert the values into the code here:

In [ ]:
im_addr = canvases[22]["images"][0]["resource"]["@id"]

line_height = 
col_width = 

start_x = 
start_y = 

lines_per_page =

Now that we have these values, we can use IIIF to get the line images.

To do this, IIIF needs two pairs of coordinates for each image region:
1. The x and y coordinates of the starting corner of the image region (normally the upper left corner)
1. The width and height of the box (always in pixels)

For the first line on the page, this is easy:
1. The x and y coordinates of the starting corner are `start_x` and `start_y`
1. The width and height of the box are `col_width` and `line_height` respectively

For all the other lines, this is slightly more complicated:
1. The width and height of the box are always `col_width` and `line_height`: this never changes.
1. The x coordinate of the starting corner is always `start_x`: this never changes either. (But what would happen if we had two columns of text?)
1. The complicated part is the y value of the starting corner of our page. For any given line of text, we need to:
   1. Take the starting y coordinate (`start_y`)
   1. Add the `line_height` once for every line of text down the page; in other words, multiply line_height by the number of the line that we are looking for (1 for the first line, 2 for the second line etc.)
   1. If the number of the line is stored in a variable `line_no`, then the value that we need is therefore `start_y + line_height * line_no`

Let's test this. As usual, we will print out the URLs first to make sure it looks right before running our code. Here is what we want to do:

1. `for` each `line_no` in the list of lines from one to the number of lines per page:
  1. Make a string containing each coordinate, separated by a comma, according to the instructions above
  1. Replace the relevant part of the URL with these coordinates (and also the 25% scaling)
  1. Print the URL so we can check that it's right
  
**Attention**: The URL needs to be a string, but our values of start_x, start_y etc. are numbers. We therefore need to tell Python to convert our numbers into strings, and we do this using the `str()` function.

Here is the code:

In [ ]:
for line_no in range(lines_per_page):
    coord = str(start_x) + "," + str(start_y + line_no*line_height) + "," + str(col_width) + "," + str(line_height)
    line_addr = im_addr.replace("/full/full/0", "/" + coord + "/pct:25/0")
    print(line_addr)

Assuming that this worked, you can now change the code to actually download the images and store them in a list. Here is the code again, as well as a new list variable. You now need to replace the `print()` function with the instructions to download the image and store it in the list.

**Hints**:
1. You will need to use `requests()`, `Image.open()`, `append()` and a couple of other methods.
1. We did almost exactly this just a little bit earlier, when we downloaded images of lines from different pages. Have another look at the code there, and you will see that you can copy and paste the necessary parts with only very small changes.

In [ ]:
line_imgs = []

for lineno in range(lines_per_page):
    coord = "/" + str(start_x) + "," + str(start_y + lineno*line_height) + "," + str(col_width) + "," + str(line_height)
    line_addr = im_addr.replace("/full/full/0", coord + "/pct:25/0")
    print(line_addr) # Change this line to download the images and add them to the line_images[] list

At this point we should have the images stored in line_imgs, which is a list. We now want to see that this worked! To do this, we need to loop through line_imgs, and `for` each image `in` our list, we should `show()` it. Type in the code here to do this (hint: it's only two lines, and once again you can copy and paste it almost exactly from an earlier example).

# Putting it Together: IIIF and XML TEI

At this point, we have seen a way of getting image regions that correspond to lines of text, and we have also seen (in earlier worksheets) how to get particular parts of XML. We can now put this together and create links between text and image.

In order to do this, we need a TEI XML file which follows the 'documentary' view, or at least that has information about lines of text. For this exercise, I have taken an extract of the _Essais_ of Michel de Montaigne, and I have restructured it into the TEI Documentary view. The result is in the file `Montaigne f22.xml` in your folder of course materials. Let's load it in to Beautiful Soup (using the template we have already seen in the TEI worksheet):

In [ ]:
from bs4 import BeautifulSoup

path = "../Example Texts/Montaigne f22.xml"

inFile = open(path)
fileXML = inFile.read()

bs = BeautifulSoup(fileXML, "xml")

inFile.close()

In order to check that this worked, let's create a list of all the `<line>` elements inside `<surface>` elements inside `<zone>` elements. We can then pick a line, print the text and display the image:

In [ ]:
linelist = bs.select("surface > zone > line")
no_lines = len(linelist)

print(linelist[8].get_text())
line_imgs[8].show()

Let's now do something a bit more interactive: ask the user for a word, and then search for that word in the text. If we find it then we display the corresponding image. Here is the algorithm:

1. Ask the user for a word
1. `For` each line number `in` 1 to the number of lines:
  1. `if` our word is in the _text_ corresponding to this line number:
    1. Display the _image_ corresponding to this line number

And here is the code:

In [ ]:
searchword = input("Enter a word ")

for lineno in range(no_lines):
    if searchword in linelist[lineno].get_text():
        line_imgs[lineno].show()

In [ ]:
image_f22 = Image.open("../Example Texts/Montaignef22_25pct.jpg")
image_f22.show()

In [ ]:
from PIL import ImageDraw

draw = ImageDraw.Draw(image_f22)

draw.rectangle([start_x, start_y, start_x + col_width, start_y + line_height], width=5)

image_f22.show()

Now, let's do something a bit more interesting. We can use this same method to draw boxes around lines of text. This means, for instance, that we could do the following:

1. Ask the user for a word
1. `For` each line of text `in` the TEI XML:
  1. Look in the text to see if that word occurs
  1. If it does occur then:
    1. Calculate the coordinates for the corresponding line of text in the image
    1. Draw a box on the image around the line

We could do this dynamically using IIIF, but it will be slow (especially with everyone trying to do it all at once). For that reason, there is a copy of the image in your folder of course materials, called Montaignef22_25pct.jpg. This is simply the image scaled down to 25% that we have been working with so far.

**Attention**: There is a catch here! We want to select the region of the image and also scale the original image to 25%. In IIIF, the standard says that _first_ you select the region, and _then_ you scale it. However, here, we have already _first_ downloaded the image at 25% and _then_ we want to select the region. This means that our coordinates are for the original image, but we need to apply these coordinates to the smaller image that is one quarter the size of the original. For this reason, we must also divide all our coordinates by four as well.

This is the code:

In [ ]:
# Open the image file and read it into an Image object
image_f22 = Image.open("../Example Texts/Montaignef22_25pct.jpg")

# Create a new Draw object to contain the boxes
draw = ImageDraw.Draw(image_f22)

# Ask the user for the word to find
searchword = input("Enter a word ")

# Loop through eacy line of text looking for the word, and drawing a box if appropriate
for lineno in range(no_lines):
    if searchword in linelist[lineno].get_text():
        draw.rectangle([start_x/4, start_y/4 + (line_height/4 * lineno), 
                        start_x/4 + col_width/4, start_y/4 + (line_height/4 * (lineno + 1))],
                                                        width=5)

# Now display the resulting image (with boxes)
image_f22.show()

# Taking it Further

As an exercise to take this further at home, try changing the code to search for the word anywhere in the document, and then use IIIF to download the appropriate image(s) and add the boxes for display. Attention, though: the XML that you have is only for one page, so you would need to restructure more of the text for this to be helpful. You could do this by hand or even automatically using Python! 

---
![Licence Creative Commons](https://i.creativecommons.org/l/by/4.0/88x31.png)
This work (the contents of this Jupyter Python notebook) is licenced under a [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)